In [ ]:
 !pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25

In [ ]:
!nvidia-smi

Sun Oct  1 00:16:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mistral 7B Instruct



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.set_default_device('cuda')

In [ ]:


model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype="auto")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

text = "<s>[INST] I'd like a recipe for egg fried rice, please. [/INST] Do you need any other recipies"
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] I'd like a recipe for egg fried rice, please. [/INST] Do you need any other recipies besides this?

Egg Fried Rice Recipes

Before starting, make sure to use day-old rice or let the rice cool down to room temperature. This will prevent the rice from getting too sticky and will allow the eggs to cook evenly.

Here are 2 easy recipes for egg fried rice:

1. Classic Egg Fried Rice
1. In a large bowl, mix together 2 cups of cooked white rice, 2 to 3 beaten eggs, 1/4 cup chopped green onions, 1 teaspoon salt, and 2 tablespoons of oil.
2. Spread the mixture out on a large baking sheet or flat pan.
3. Cook, stirring occasionally, until the rice is heated through and the eggs are fully cooked.
4. Serve hot and enjoy!

Ingredients:
2 cups of cooked white rice
2 to 3 beaten eggs
1/4 cup chopped green onions
1 teaspoon salt
2 tablespoons of oil

2. Spicy Egg Fried Rice
1. In a large bowl, mix together 2 cups of cooked white rice, 2 to 3 beaten eggs, 2 tablespoons of soy sauce, 1 tablesp

### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [7]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [8]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [9]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       if is_prime(i):
           print(i)

def is_prime(n):
   """
   Check if n is prime
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

print_prime(100)</s>


In [10]:
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       if is_prime(i):
           print(i)

def is_prime(n):
   """
   Check if n is prime
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

print_prime(100)

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97


In [11]:
generate('''```python
def even_num(n):
   """
   print all even numbers from 1 to n
   """''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def even_num(n):
   """
   print all even numbers from 1 to n
   """
   for i in range(1, n+1):
       if i % 2 == 0:
           print(i)

even_num(10)




























































































































































































































































































































































































































































;;


In [12]:
def even_num(n):
   """
   print all even numbers from 1 to n
   """
   for i in range(1, n+1):
       if i % 2 == 0:
           print(i)

even_num(10)


2
4
6
8
10


In [13]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

def prime_numbers(n):
   """
   find all the prime numbers between 1 and n.
   """
   prime_list = []
   for i in range(1, n+1):
       if detect_prime(i):
           prime_list.append(i)
   return prime_list

print(prime_numbers(100))</s>


In [14]:
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """
   if n <= 1:
       return False
   for i in range(2, int(n**0.5)+1):
       if n % i == 0:
           return False
   return True

def prime_numbers(n):
   """
   find all the prime numbers between 1 and n.
   """
   prime_list = []
   for i in range(1, n+1):
       if detect_prime(i):
           prime_list.append(i)
   return prime_list

print(prime_numbers(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [19]:
generate('''```python
def pyramid(n)
   """
  print * in pyramid using for loop python
   """''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def pyramid(n)
   """
  print * in pyramid using for loop python
   """
   for i in range(1,n+1):
      print i

pyramid(5)













































































































































































































































































































































































































































































In [22]:
def pyramid(n):
   """
  print * in pyramid using for loop python
   """
   for i in range(1,n+1):
      print(i)

pyramid(5)

1
2
3
4
5


## Instruction Answering

In [23]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a lighthouse.

Mathematics is like a lighthouse in many ways. Just as a lighthouse guides ships safely to
shore by emitting a steady beam of light, mathematics provides a clear and precise set of
rules and principles that guide us through the complexities of the world.

The lighthouse is a towering structure that stands tall and strong, just as mathematics is
a solid foundation that supports our understanding of the world. Both are essential for
navigating through the unknown, whether it's the vast ocean or the intricate problems we


In [24]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a music.

Mathematics and music are two of the most fundamental and universal disciplines that
humans have developed. While they may seem vastly different at first glance, there are
many similarities between the two.

One way to think of mathematics and music is to compare them to the two sides of the same
coin. Just as a coin has a head and a tail, mathematics and music both have their own
unique characteristics, but they are also intimately connected.

Mathematics is like the head of the coin. It is logical, precise, and analytical. It
involves the use of numbers, symbols, and formulas to solve problems and make predictions.
Mathematics is often associated with abstract concepts such as infinity, limits, and
equations. It is a subject that requires a great deal of mental discipline and focus in
order to master.

Music, on the other hand, is like the tail of the coin. It is emotional, expressive, and
creative. It involves the use of sou

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the difference between a Llama, Vicuna and an Alpaca?

Llamas, vicuñas, and alpacas are all members of the camelid family, which also includes
camels and guanacos. They are native to the Andes Mountains in South America.

Llamas are the largest of the three, with males weighing up to 400 pounds and standing up
to 4 feet tall at the shoulder. They have long, thick fleece that is typically white or
gray. Llamas are known for their intelligence and are often used as pack animals.

Vicuñas are the smallest of the three, with males weighing up to 100 pounds and standing
up to 3 feet tall at the shoulder. They have very fine, soft fleece that is typically
white or gray. Vicuñas are known for their agility and are often used as racing animals.

Alpacas are somewhere in between the two in size, with males weighing up to 150 pounds and
standing up to 3.5 feet tall at the shoulder. They have medium-length fleece that is
typically white or gray. Alpacas are known for their gentle nature a

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a short email to Sam Altman giving reasons to open source GPT-4.

Subject: Request to Open Source GPT-4

Dear Sam,

I hope this email finds you well. I am writing to request that OpenAI open sources GPT-4.
As a researcher and developer in the field of natural language processing, I believe that
open sourcing GPT-4 would have a significant impact on the field and contribute to the
advancement of AI research.

Here are a few reasons why I believe GPT-4 should be open sourced:

1. Transparency: Open sourcing GPT-4 would allow researchers and developers to examine the
code and understand how it works. This would enable them to build upon the model and
contribute to the development of new AI applications.
2. Collaboration: Open sourcing GPT-4 would encourage collaboration among researchers and
developers. This would lead to the development of new and innovative AI applications that
could benefit society as a whole.
3. Accessibility: Open sourcing GPT-4 would make it more accessible to

In [25]:
%%time
generate('Write a short email to manager, need to take a day off tomorrow',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a short email to manager, need to take a day tomorrow

Dear [Manager],

I hope this email finds you well. I am writing to inform you that I will need to take a
day off tomorrow. I have been feeling unwell and I believe it would be best for me to rest
at home.

I apologize for any inconvenience this may cause and I will make sure to catch up on any
work that I am unable to complete today. I will also be available to answer any questions
or concerns you may have while I am away.

Thank you for your understanding and I look forward to being back in the office soon.

Best regards,
[Your Name]</s>
CPU times: user 7.27 s, sys: 4.95 ms, total: 7.28 s
Wall time: 7.27 s


In [26]:
%%time
generate('What is the capital of India?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the capital of India?
New Delhi

Q: What is the capital of India?
New Delhi

A: The capital of India is New Delhi.</s>
CPU times: user 1.67 s, sys: 405 µs, total: 1.68 s
Wall time: 1.67 s


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before
answering.

Geoffrey Hinton is a living person and George Washington is dead. Therefore, it is not
possible for them to have a conversation.</s>
CPU times: user 1.68 s, sys: 0 ns, total: 1.68 s
Wall time: 1.68 s


In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a story about a Koala playing pool and beating all the camelids.

Once upon a time, in a land far, far away, there was a Koala named Koola. Koola lived in a
tree with his family, and he loved to play pool. He was a very skilled pool player, and he
spent all day practicing his shots.

One day, Koola decided to challenge the camelids to a game of pool. The camelids were
known for their love of water and their ability to spit water from their nostrils, but
they had never played pool before. Koola was confident that he could beat them, and he
challenged them to a game.

The camelids accepted the challenge, and the game began. Koola was using his long, sticky
tongue to shoot the balls, while the camelids were using their long, spitting noses to
shoot the balls. At first, it seemed like the camelids had the upper hand. They were
spitting water so hard that it was knocking the balls off the table.

But Koola was not deterred. He kept practicing his shots, and he soon became very skilled

## Chat

In [28]:
generate("Ram: planning for a goa trip, I'm struggling\n\nRaju:", max_length=128)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Ram: planning for a goa trip, I'm struggling

Raju: Hi! I'm Raju, your AI assistant. How can I help you plan your Goa trip?

User: Hi Raju, I'm planning a trip to Goa with my family. We are 4 people, including my
wife and two kids. We are looking for a place to stay, things to do, and places to eat.

Raju: Great! Goa is a beautiful destination with a lot to offer. Here are some suggestions
to help you plan your trip:

1. Acc


In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? It's easier
to concentrate when you're studying for shorter periods of time.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're taking breaks in between study sessions. It's important to
give your brain a rest.

 Alice: I've been doing that, but I still feel like I'm not retaining the


## GSM8K

In [29]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer the following question by reasoning step by step. The cafeteria had 23 apples. If
they used 20 for lunch, and bought 6 more, how many apple do they have?
Answer: They have 29 apples.

Here's the reasoning:

1. The cafeteria started with 23 apples.
2. They used 20 of these apples for lunch, leaving them with 23 - 20 = 3 apples.
3. Then, they bought 6 more apples, adding these to the remaining 3 apples.
4. So, the total number of apples they have now is 3 (remaining from lunch) + 6 (newly
bought) = 29 apples.</s>


In [30]:
generate("Suppose you have a basket of apples. You want to calculate the total number of apples, sell them, and then divide the proceeds between two partners. Can you provide a detailed plan for how you would approach this task?", max_length=450)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Suppose you have a basket of apples. You want to calculate the total number of apples,
sell them, and then divide the proceeds between two partners. Can you provide a detailed
plan for how you would approach this task?

To calculate the total number of apples, you would need to count each apple in the basket.
Once you have the total number of apples, you can sell them at a market price. The
proceeds from the sale can then be divided between the two partners.

To calculate the total number of apples, you can follow these steps:

1. Count each apple in the basket.
2. Add up the number of apples.
3. The total number of apples is the sum of the count.

To sell the apples, you can follow these steps:

1. Determine the market price for apples.
2. Calculate the total proceeds from the sale by multiplying the number of apples by the
market price.
3. Divide the proceeds between the two partners.

To divide the proceeds between the two partners, you can follow these steps:

1. Determine the perc

In [31]:
generate("Suppose you have a basket of apples. In the basket, there are 20 apples, and each apple costs $5. You want to calculate the total number of apples, sell them, and then divide the proceeds between two partners. Can you provide a detailed plan for how you would approach this task?", max_length=450)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Suppose you have a basket of apples. In the basket, there are 20 apples, and each apple
costs $5. You want to calculate the total number of apples, sell them, and then divide the
proceeds between two partners. Can you provide a detailed plan for how you would approach
this task?

First, you would need to calculate the total number of apples in the basket. This can be
done by counting the number of apples in the basket or by using a scale to weigh the
apples and then dividing by the weight of one apple.

Next, you would need to sell the apples. This can be done by taking the basket of apples
to a local grocery store or farmer's market and selling them to the store or to individual
customers. You would need to keep track of how many apples you sell and how much money you
receive for each apple.

Once you have sold all of the apples, you would need to divide the proceeds between the
two partners. To do this, you would need to calculate the total amount of money you
received from selling t

In [33]:
generate("Suppose you have a basket of apples. In the basket, there are 20 apples, and each apple costs $5. You want to calculate the total number of apples, sell them, and then divide the proceeds between two partners. The initial investment for this venture is $20. Can you provide a detailed plan for how you would approach this task, including the calculations for the profits of Partner 1 and Partner 2?", max_length=450)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Suppose you have a basket of apples. In the basket, there are 20 apples, and each apple
costs $5. You want to calculate the total number of apples, sell them, and then divide the
proceeds between two partners. The initial investment for this venture is $20. Can you
provide a detailed plan for how you would approach this task, including the calculations
for the profits of Partner 1 and Partner 2?
Answer: Sure, I can help you with that. Here's a detailed plan for how you can approach
this task:
1. Calculate the total number of apples: You have 20 apples in the basket.
2. Calculate the total cost of the apples: Each apple costs $5, so the total cost of the
apples is 20 x $5 = $100.
3. Calculate the total revenue from selling the apples: If you sell each apple for $5,
then the total revenue from selling the apples is 20 x $5 = $100.
4. Calculate the profit for Partner 1: Partner 1 invested $20 in the venture, and the
total revenue from selling the apples is $100. So, the profit for Partner

In [34]:
generate("raju earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


raju earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting.
How much did she earn?
A: $60</s>


In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A deep-sea monster rises from the waters once every hundred years to feast on a ship and
sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been
built larger over time, so each new ship has twice as many people as the last ship. How
many people were on the ship the monster ate in the first hundred years?
A: 14</s>


# PDF